In [1]:
import pandas as pd
import gzip
import Bio
from Bio import SeqIO
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq
import pickle
from collections import Counter
import timeit
import copy
###
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import igraph
import seaborn
###import packages
from scEasyMode import mousefilter
from scEasyMode import clusterplot
from scEasyMode import pymulti
from scEasyMode import sceasy

In [2]:
import pkg_resources

# Retrieve and print the version of all packages currently loaded in the environment
for pkg in pkg_resources.working_set:
    print(f"{pkg.key}=={pkg.version}")

zipp==0.6.0
wheel==0.33.4
webencodings==0.5.1
wcwidth==0.1.7
urllib3==1.25.3
umap-learn==0.3.10
traitlets==4.3.3
tornado==5.1.1
testpath==0.4.2
terminado==0.8.2
tables==3.4.4
statsmodels==0.9.0
sklearn==0.0
six==1.12.0
setuptools==41.0.1
send2trash==1.5.0
seaborn==0.9.0
scipy==1.2.1
scikit-learn==0.20.2
scanpy==1.4
s3transfer==0.2.1
ruamel-yaml==0.15.46
rsa==3.4.2
requests==2.21.0
pyzmq==18.1.0
pyyaml==5.1
pytz==2018.9
python-levenshtein==0.12.0
python-igraph==0.7.1.post6
python-dateutil==2.8.0
pysocks==1.7.0
pyrsistent==0.15.4
pyparsing==2.3.1
pyopenssl==19.0.0
pygments==2.4.2
pycparser==2.19
pycosat==0.6.3
pycairo==1.18.0
pyasn1==0.4.6
ptyprocess==0.6.0
prompt-toolkit==2.0.10
prometheus-client==0.7.1
pip==19.2.1
pickleshare==0.7.5
pexpect==4.7.0
patsy==0.5.1
parso==0.5.1
pandocfilters==1.4.2
pandas==0.24.1
numpy==1.15.4
numexpr==2.6.9
numba==0.42.0
notebook==6.0.1
networkx==2.2
nbformat==4.4.0
nbconvert==5.6.0
natsort==6.0.0
more-itertools==7.2.0
mkl-random==1.0.2
mkl-fft==1.0.10
mis

## check files

In [ ]:
%%bash 

cd bulkgbcs
ls -hlt

In [ ]:
## exclude L0130 because it is way undersampled.

## readin raw files and split

In [ ]:
#####define files
#######note that R1 = R2 , vice versa. therefore the important 10x 3' read is in read1_list
barcodes = [120,121,122,123,124,125,126,127,128,129,130,131]
reads = []
for barcode in barcodes:
    read2 = "bulkgbcs/L0"+str(barcode)+".R2.fastq.gz"
    tmpread =[]
    ##
    with gzip.open(read2,"rt") as handle:
        for record in SeqIO.parse(handle,'fastq'):
            tmpread.append(str(record.seq[:18]))
    ##
    reads.append(tmpread)

In [ ]:
#######save read data
import  csv

with open("bulkgbcs.raw.csv","w") as f:
    wr = csv.writer(f)
    wr.writerows(reads)

## read in split read data

In [2]:
import csv 
import sys 
file_name = 'bulkgbcs.raw.csv'
with open(file_name, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    reads = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists


/avicenna/Johnny/envs/scanpy2/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.


In [ ]:
# ### removal of number L0130
# reads.remove(reads[10])

## stack and count

In [3]:
#######count reads
counts = []
for read in reads:
    counts.append(np.unique(read,return_counts=True))

In [4]:
def hamming(a, b):
    return len([i for i in filter(lambda x: x[0] != x[1], zip(a, b))])

## get valid barcodes based on hamming distance

In [5]:
###
FDR_barcodes = []
for i in range(len(counts)):
    FDR = 0.0
    ####
    for pct in np.arange(99,20,-0.5):
        if FDR < .05:
            #######filter sequentially by FDR until FDR threshold reached
            thresh = np.percentile(counts[i][1],pct)
            filtered = pd.DataFrame(counts[i][1],counts[i][0]).sort_values(by=0)
            filtered = filtered[filtered[0]>=thresh]
            print("#barcodes:",len(filtered))
            print('percentile:',pct)
            print("thresh:",thresh)
            ####
            testbc = filtered.index.tolist()
            testcount = filtered[0].tolist()
            dists=[]
            barcodes = []
            bccounts = []
            ##
            fp_barcodes = []
            ###
            for item1,count1 in zip(testbc,testcount):
                tmpdist = []
                tmpcount = []
                tmpbarcode = []
                for item2,count2 in zip(testbc,testcount):
                    hamm = hamming(item1,item2)
                    ###decide which one to keep which one to remove
                    if hamm == 1: 
                        if count1 < count2:
                            fp_barcodes.append(item1)
                        else:
                            fp_barcodes.append(item2)
                    ####
                    tmpdist.append(hamm)
                    tmpbarcode.append([item1,item2])
                    tmpcount.append([count1,count2])
                ######
                dists.append(tmpdist)
                barcodes.append(tmpbarcode)
                bccounts.append(tmpcount)
            ####
            fp_barcodes = list(set(fp_barcodes))
            if len(fp_barcodes)!= 0:
                if len(testbc)!=0:
                    FDR = len(fp_barcodes)/len(testbc)
                    print("FDR:",FDR)
                    print("dataset:",i)
                    ###
                    if FDR >0.05:
                        FDR_barcodes.append([testbc,fp_barcodes])


#barcodes: 204
percentile: 99.0
thresh: 9908.079999999987
#barcodes: 306
percentile: 98.5
thresh: 4161.209999999999
FDR: 0.0032679738562091504
dataset: 0
#barcodes: 408
percentile: 98.0
thresh: 685.5599999999977
FDR: 0.0024509803921568627
dataset: 0
#barcodes: 512
percentile: 97.5
thresh: 32.0
FDR: 0.078125
dataset: 0
#barcodes: 315
percentile: 99.0
thresh: 4563.479999999996
FDR: 0.0031746031746031746
dataset: 1
#barcodes: 472
percentile: 98.5
thresh: 169.5949999999939
FDR: 0.012711864406779662
dataset: 1
#barcodes: 638
percentile: 98.0
thresh: 38.0
FDR: 0.1865203761755486
dataset: 1
#barcodes: 231
percentile: 99.0
thresh: 9028.200000000168
FDR: 0.004329004329004329
dataset: 2
#barcodes: 347
percentile: 98.5
thresh: 2787.4800000000105
FDR: 0.002881844380403458
dataset: 2
#barcodes: 462
percentile: 98.0
thresh: 235.87999999999738
FDR: 0.004329004329004329
dataset: 2
#barcodes: 577
percentile: 97.5
thresh: 28.099999999998545
FDR: 0.1195840554592721
dataset: 2
#barcodes: 303
percentile: 9

In [6]:
cleanbcs = []
####
for bcs in FDR_barcodes:
    totalbcs = bcs[0]
    fpbcs = bcs[1]
    tmpclean = []
    ###
    for bc in totalbcs:
        if bc not in fpbcs:
            tmpclean.append(bc)
    ###
    print(len(bcs[0]),len(bcs[1]),len(tmpclean))
    cleanbcs.append(tmpclean)

512 40 472
638 119 519
577 69 508
609 83 526
534 28 506
533 37 496
601 60 541
681 120 561
612 93 519
557 30 527
526 35 491
496 32 464


## Visualize results of barcode calling
#### see amount of overlap between datapoints


In [ ]:
## get union
set_union = set(cleanbcs[0]).union(*cleanbcs)

In [ ]:
## find union barcodes from fdr discovery 
clean_counts = []
###subset
for i in range(len(counts)):
    filtered = pd.DataFrame(counts[i][1],counts[i][0]).sort_values(by=0)
    filtered = filtered[filtered.index.isin(set_union)]
    clean_counts.append(filtered/filtered[0].sum())

In [ ]:
merged = pd.concat(clean_counts,axis=1,sort=True)
merged.columns = ['ctl_2','t_0','osi_2','t_0','ctl_2','osi_9','osi_2','ctl_2','osi_2','t_0','osi_9','osi_9']
###
merged = merged.transpose().sort_index()


## save merged proportions

In [ ]:
merged.to_csv('bulk.merged.gbcs.csv',sep=',')

In [ ]:
test = merged.replace(to_replace=np.nan,value=-100)
seaborn.clustermap(test,row_cluster=False)

## read in merged proportions

In [ ]:
merged = pd.read_csv('bulk.merged.gbcs.csv',sep=',',index_col=0)
merged

## only keep barcodes that are present in all three T0 

In [ ]:
t_0 = merged.transpose()['t_0']
t_0 = t_0.dropna()
### define whitelist
whitelist = t_0.index.tolist()

In [ ]:
test = merged[whitelist]
test = test.replace(to_replace=np.nan,value=-100)
seaborn.clustermap(test,row_cluster=False)

## finer visualizations

In [ ]:
### divide by T0 , scale to 1
test = merged[whitelist]
test = test.replace(to_replace=np.nan,value=0)
test = test/test.loc['t_0'].mean()
### a lot outliers, scale
test[test>1.1] = 1.1
seaborn.clustermap(test,vmin=0,row_cluster=False)

In [ ]:
test

In [ ]:
test

## save

In [ ]:
pd.DataFrame(whitelist).to_csv('gbcs.cleanintx.csv')

In [ ]:
clean_counts = []
###subset
for i in range(len(counts)):
    filtered = pd.DataFrame(counts[i][1],counts[i][0]).sort_values(by=0)
    filtered = filtered[filtered.index.isin(clean_intersect)]
    clean_counts.append(filtered/filtered[0].sum())

In [ ]:
merged = pd.concat(clean_counts,axis=1)
merged.columns = ['ctl_2','t_0','osi_2','t_0','ctl_2','osi_9','osi_2','ctl_2','osi_2','t_0','osi_9','osi_9']

In [ ]:
keep = []
for x in merged.columns: 
    if x[:3] == 'ctl':
        keep.append(False)
    else:
        keep.append(True)
####
OSI = merged.columns[keep]
OSI = list(set(OSI))
OSI = merged[OSI]

In [ ]:
OSI['T0'] = OSI['t_0'].mean(axis=1)
OSI['T2'] = OSI['osi_2'].mean(axis=1)
OSI['T9'] = OSI['osi_9'].mean(axis=1)
###
calcs = OSI.copy()

In [ ]:
import scipy.stats as stats
###
calcs['T2_pval'] = calcs.apply(lambda row: -np.log10(stats.ttest_ind(row['t_0'],row['osi_2'])[1]),axis=1)
calcs['T9_pval'] = calcs.apply(lambda row: -np.log10(stats.ttest_ind(row['t_0'],row['osi_9'])[1]),axis=1)
###
calcs['T2_log2fc'] = calcs.apply(lambda row: np.log2(row['T2']/row['T0']),axis=1)
calcs['T9_log2fc'] = calcs.apply(lambda row: np.log2(row['T9']/row['T0']),axis=1)

In [ ]:
calcs.plot.scatter(x='T2_log2fc',y='T2_pval')
calcs.plot.scatter(x='T9_log2fc',y='T9_pval')


In [ ]:
log2fc = calcs[['T2_log2fc','T9_log2fc']]
seaborn.clustermap(log2fc,col_cluster=False,cmap='mako')

In [ ]:
tmp = pd.DataFrame(calcs[['T2_log2fc','T9_log2fc']].stack())
tmp = tmp.reset_index(level=1)
tmp.columns = ['timepoint','log2fc']
seaborn.violinplot(y='timepoint',x='log2fc',data=tmp,scale='area',gridsize=300,cut=0)
plt.savefig('figures/log2fc_by_barcodes.pdf')
